In [1]:
from superneuromat.neuromorphicmodel import NeuromorphicModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field


# from: https://stackoverflow.com/a/21894086/2712730
class bidict(dict):
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value, []).append(key)

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key)
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value, []).append(key)

    def __delitem__(self, key):
        self.inverse.setdefault(self[key], []).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]:
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)


def train_test_split_indices(papers, test_size=0.2, rng=None) -> tuple[np.ndarray[int], np.ndarray[int]]:
    """
    Splits a list of papers into train and test indices.

    Parameters
    ----------
    papers : iterable of papers with ids
        List or dict of papers to split.
    test_size : float, optional
        if < 1, proportion of papers to reserve for testing, by default 0.2
        if > 1, number of papers to reserve for testing
    rng : int, optional
        Random state for the random number generator, default uses numpy's random

    Returns
    -------
    train_indices : list of int
        List of indices for the training set.
    test_indices : list of int
        List of indices for the testing set.
    """
    n = papers if isinstance(papers, int) else len(papers)
    if test_size < 1:
        test_size = int(np.floor(test_size * n))  # number of papers in test
    if rng is None:
        rng = np.random.default_rng()  # setup rng
    elif isinstance(rng, int):
        rng = np.random.default_rng(rng)

    if isinstance(papers, int):
        indices = np.arange(n)  # generate indices
    elif isinstance(papers, (list, tuple)):
        indices = papers  # assume papers is a list of indices
    else:  # assume papers is a dict or mapping of papers with a .values() method
        indices = [paper.idx for paper in papers.values()]  # grab indices from dict entries
    indices = np.asarray(indices, dtype=int)

    # shuffle and split
    rng.shuffle(indices)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return train_indices, test_indices


In [2]:
rng = np.random.default_rng(2024)

In [3]:
@dataclass
class Paper:
    idx: int
    label: str
    features: tuple[bool, ...] = ()
    citations: list[int] = field(default_factory=list)

papers = {}

# Load in training data
content = pd.read_csv("data/Cora/cora/cora.content", sep="\t", header=None)
citations = pd.read_csv("data/Cora/cora/cora.cites", sep="\t", header=None)

labels = set()

for paper in content.itertuples(index=False):
    idx = paper[0]
    features = tuple([int(feature) for feature in paper[1:-1]])
    papers[idx] = Paper(idx, paper[-1], features)
    labels.add(paper[-1])

for paper_idx, citation  in citations.itertuples(index=False):
    papers[paper_idx].citations.append(citation)

labels

{'Case_Based',
 'Genetic_Algorithms',
 'Neural_Networks',
 'Probabilistic_Methods',
 'Reinforcement_Learning',
 'Rule_Learning',
 'Theory'}

In [4]:
train_idxs, test_idxs = train_test_split_indices(papers, test_size=0.2, rng=rng)

In [5]:
# Initialize our model
model = NeuromorphicModel()

# Create our output neurons, set threshold very high so that we control when they spike during training.
# dict mapping {category: neuron_id}
lbl_neurons = bidict({label: model.create_neuron(threshold=10000) for label in labels})

# Create our input neurons, one for each pixel of the image resolution.
paper_neurons = {}  # dict mapping {paper_id: neuron_id}

# make a neuron for each paper
for paper in papers.values():
    paper_neurons[paper.idx] = neuron_id = model.create_neuron()

    if paper.idx in train_idxs:
        # Make an explicitly STRONG synapse connecting the input to the output
        output_id = lbl_neurons[paper.label]
        model.create_synapse(neuron_id, output_id, weight=10001, enable_stdp=False, delay=1)
    else:  # test set, connect this input neuron to all output neurons
        # Connect our input neuron to output neurons
        for output_id in lbl_neurons.values():
            # Randomize initial weight
            weight = (rng.uniform() * 2 - 1) * 0.00001
            # Make a synapse connecting the input to the output
            model.create_synapse(neuron_id, output_id, weight=weight, enable_stdp=True, delay=1)

# connect papers by their citations
for paper in papers.values():
    for citation in paper.citations:
        model.create_synapse(paper_neurons[paper.idx], paper_neurons[citation], weight=1.0, enable_stdp=True, delay=1)

In [6]:
# loop through our dataset and add spikes
timestep = 0
train_idxs_augmented = np.append(train_idxs, train_idxs)
for idx in train_idxs_augmented:
    paper = papers[idx]  # get paper by id
    # Add a spike that will exceed the threshold for the respective label neuron
    model.add_spike(timestep + 1, lbl_neurons[paper.label], 10001.0)
    # Add spikes to the paper
    model.add_spike(timestep, paper_neurons[paper.idx], 10001.0)
    timestep += 1

In [7]:
# Set up our stdp, only one timestep because we only want it looking at the results
# of what our input layer does to our output layer
model.stdp_setup(time_steps=1, Apos=[0.001], Aneg=[0.000001], negative_update=True, positive_update=True)
model.setup()

print(f"Your model has {model.num_neurons} neurons and {model.num_synapses} synapses.\n{timestep} time steps will be simulated.")

import tqdm

# Simulate
with tqdm.tqdm(total=timestep) as pbar:
    model.simulate(time_steps=timestep, callback=lambda _s, _t, _n: pbar.update(), use='jit')

Your model has 2715 neurons and 11383 synapses.
4334 time steps will be simulated.


100%|██████████| 4334/4334 [02:59<00:00, 24.17it/s]


### inference

In [8]:
def blank_copy(model):
    new = NeuromorphicModel()
    new.num_neurons = model.num_neurons
    new.neuron_thresholds = model.neuron_thresholds.copy()
    new.neuron_leaks = model.neuron_leaks.copy()
    new.neuron_reset_states = model.neuron_reset_states.copy()
    new.neuron_refractory_periods = model.neuron_refractory_periods.copy()
    new.num_synapses = model.num_synapses
    new.pre_synaptic_neuron_ids = model.pre_synaptic_neuron_ids.copy()
    new.post_synaptic_neuron_ids = model.post_synaptic_neuron_ids.copy()
    new.synaptic_weights = model.synaptic_weights.copy()
    new.synaptic_delays = model.synaptic_delays.copy()
    new.enable_stdp = model.enable_stdp.copy()
    return new

In [9]:
seed = 10022

In [10]:
seed += 1
print(f"Seed: {seed}")
rng = np.random.default_rng(seed)
model2 = blank_copy(model)
model2.enable_stdp = np.zeros(model.num_neurons)
model2.neuron_thresholds[:7] = [50] * 7

# paper = random.choice(list(papers.values()))  # pick a random paper
# paper = papers[random.choice(train_idxs)]  # pick a random paper from training set
paper = papers[rng.choice(test_idxs)]  # pick a random paper from testing set

print(f"Paper: {paper.idx}\tCategory: {paper.label}")

spike_n = 2

for t in range(spike_n):
    model2.add_spike(t, paper_neurons[paper.idx], 10001.)

model2.setup()
model2.simulate(spike_n + 1, use='jit')

spiked_ids = {idx: lbl_neurons.inverse[idx][0]
              for idx, spiked in enumerate(model2.spike_train[-1][:7]) if spiked}
if spiked_ids:
    print(f"Spiked categories:")
    for idx, category in spiked_ids.items():
        print(f"\t{idx}\t{category}")
else:
    print("No category spiked")

print('=' * 10)

lbl_by_threshold = sorted((enumerate(model2._internal_states[:7])), key=lambda x: x[1], reverse=True)
for i, v in lbl_by_threshold:
    category = lbl_neurons.inverse[i][0]
    print(f"{i}\t{v: 5.3f}\t{category}")
print(model2._internal_states[:7])
print(model2._neuron_thresholds[:7])
for ts in model2.spike_train:
    print(ts[:7])

Seed: 10023
Paper: 1130847	Category: Genetic_Algorithms
No category spiked
1	 4.323	Reinforcement_Learning
3	 4.323	Rule_Learning
4	 4.323	Probabilistic_Methods
6	 4.323	Neural_Networks
5	 4.323	Case_Based
2	 4.323	Theory
0	 4.323	Genetic_Algorithms
[4.3229814  4.3229988  4.322986   4.3229969  4.32299476 4.32298739
 4.32299147]
[50. 50. 50. 50. 50. 50. 50.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0.]


In [11]:
def evaluate_paper(paper_idx):
    model_temp = blank_copy(model)
    model_temp.enable_stdp = np.zeros(model.num_neurons)
    model_temp.neuron_thresholds[:7] = [50] * 7
    for t in range(2):
        model_temp.add_spike(t, paper_neurons[paper_idx], 10001.)
    model_temp.setup()
    model_temp.simulate(len(test_idxs) + 2, use='jit')
    return model_temp.spike_train[-1][:7]

In [ ]:
results = [evaluate_paper(paper_idx) for paper_idx in tqdm.tqdm(test_idxs)]

In [27]:
tp = 0
correct = 0
total = len(test_idxs)
fp = total
tn = 0
fn = 0

for actual_idx, spikes in zip(test_idxs, results):
    correct_label = papers[actual_idx].label
    guesses = [lbl_neurons.inverse[idx][0] for idx, spiked in enumerate(spikes) if spiked]
    if correct_label in guesses:
        tp += 1
        fp -= 1
        if len(guesses) == 1:
            correct += 1
    elif guesses:
        fp -= 1
    else:
        fn += 1

n_guesses = total - fp
print(f"Okay guesses: {tp} / {n_guesses} / {total} | Perfect guesses: {correct} / {n_guesses} / {total} (correct / attempted / total)")
print(f"Okay guesses: {tp / n_guesses:>.9f} | Perfect guesses: {correct / n_guesses:>.9f}")
print(f"Precision:    {tp / (tp + fp):>.9f} | Accuracy:        {(tp + tn) / (tp + tn + fp + fn):>.9f}")


Okay guesses: 119 / 130 / 541 | Perfect guesses: 76 / 130 / 541 (correct / attempted / total)
Okay guesses: 0.915384615 | Perfect guesses: 0.584615385
Precision:    0.224528302 | Accuracy:        0.126461211
